# SemaDB

> [SemaDB](https://www.semafind.com/products/semadb) 来自 [SemaFind](https://www.semafind.com)，它是一个简洁的向量相似性数据库，用于构建 AI 应用。托管的 `SemaDB Cloud` 提供了无忧的开发者体验，方便入门。

完整的 API 文档、示例和交互式游乐场可在 [RapidAPI](https://rapidapi.com/semafind-semadb/api/semadb) 上找到。

本笔记展示了 `SemaDB Cloud` 向量存储的使用方法。

您需要运行 `pip install -qU langchain-community` 来安装 `langchain-community` 以使用此集成。

## 加载文档嵌入

要在本地运行，我们使用 [Sentence Transformers](https://www.sbert.net/)，这是一种常用的句子嵌入模型。您可以使用 LangChain 提供的任何嵌入模型。

In [ ]:
%pip install --upgrade --quiet  sentence_transformers

In [2]:
from langchain_huggingface import HuggingFaceEmbeddings

model_name = "sentence-transformers/all-mpnet-base-v2"
embeddings = HuggingFaceEmbeddings(model_name=model_name)

In [3]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter

loader = TextLoader("../../how_to/state_of_the_union.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=400, chunk_overlap=0)
docs = text_splitter.split_documents(documents)
print(len(docs))

114


## 连接到 SemaDB

SemaDB Cloud 使用 [RapidAPI 密钥](https://rapidapi.com/semafind-semadb/api/semadb) 进行身份验证。您可以通过创建免费的 RapidAPI 账户来获取您的密钥。

In [4]:
import getpass
import os

if "SEMADB_API_KEY" not in os.environ:
    os.environ["SEMADB_API_KEY"] = getpass.getpass("SemaDB API Key:")

SemaDB API Key: ········


In [5]:
from langchain_community.vectorstores import SemaDB
from langchain_community.vectorstores.utils import DistanceStrategy

SemaDB 向量存储的参数直接 반영 了 API：

*   "mycollection"：我们将在此存储向量的集合名称。
*   768：向量的维度。在我们的示例中，sentence transformer 嵌入会产生 768 维的向量。
*   API_KEY：你的 RapidAPI 密钥。
*   embeddings：对应着文档、文本和查询的嵌入将如何生成。
*   DistanceStrategy：使用的距离度量。如果使用 COSINE，包装器会自动标准化向量。

In [6]:
db = SemaDB("mycollection", 768, embeddings, DistanceStrategy.COSINE)

# Create collection if running for the first time. If the collection
# already exists this will fail.
db.create_collection()

True

SemaDB 矢量存储包装器会将文档文本作为点元数据添加以便之后收集。*不建议* 存储大块文本。如果您要索引大型集合，我们建议存储文档的引用，例如外部 ID。

In [7]:
db.add_documents(docs)[:2]

['813c7ef3-9797-466b-8afa-587115592c6c',
 'fc392f7f-082b-4932-bfcc-06800db5e017']

## 相似性搜索

我们使用默认的 LangChain 相似性搜索接口来搜索最相似的句子。

In [8]:
query = "What did the president say about Ketanji Brown Jackson"
docs = db.similarity_search(query)
print(docs[0].page_content)

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.


In [9]:
docs = db.similarity_search_with_score(query)
docs[0]

(Document(page_content='And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.', metadata={'source': '../../how_to/state_of_the_union.txt', 'text': 'And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.'}),
 0.42369342)

## 清理

您可以删除该 collection 来移除所有数据。

In [10]:
db.delete_collection()

True